# Binary encoding

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, DepthwiseConv2D
import tensorflow as tf

In [2]:
# Opciones de decodificación para otros parámetros
layer_type_options = {
    0: 'Conv2D', 
    1: 'BatchNorm', 
    2: 'MaxPooling', 
    3: 'Dropout', 
    4: 'Dense', 
    5: 'Flatten',
    6: 'DepthwiseConv2D',  
    7: 'DontCare',  
    8: 'Repetition'
}
stride_options = {0: 1, 1: 2}
dropout_options = {0: 0.2, 1: 0.3, 2: 0.4, 3: 0.5}
activation_options = {0: 'relu', 1: 'leaky_relu', 2: 'sigmoid', 3: 'tanh'}

# Función para codificar los parámetros de la capa
def encode_layer_params(layer_type_idx, param1=0, param2=0, param3=0):
    """
    Codifica una capa en una lista en función del tipo de capa y sus parámetros.
    
    layer_type_idx : int : índice del tipo de capa según layer_type_options.
    param1         : int/float : filtros, neuronas, capas de repetición, etc.
    param2         : int : stride, número de repeticiones, etc.
    param3         : int : índice de activación o tasa de dropout.
    """
    return [layer_type_idx, param1, param2, param3]

# Función para decodificar los parámetros de la capa
def decode_layer_params(encoded_params):
    """
    Decodifica una capa desde su representación codificada en parámetros interpretables.
    
    encoded_params : list : [tipo de capa, param1, param2, param3].
    """
    layer_type_idx = encoded_params[0]
    layer_type = layer_type_options.get(layer_type_idx, 'DontCare')
    
    # Decodificar en función del tipo de capa
    if layer_type in ['Conv2D', 'DepthwiseConv2D']:
        filters = max(4, min(encoded_params[1], 32))  # Limitar filtros entre 4 y 32
        strides = stride_options.get(encoded_params[2], 1)
        activation = activation_options.get(encoded_params[3], 'relu')
        return {
            'type': layer_type,
            'filters': filters,
            'strides': strides,
            'activation': activation
        }
    elif layer_type == 'BatchNorm':
        return {'type': 'BatchNorm'}
    elif layer_type == 'MaxPooling':
        strides = stride_options.get(encoded_params[1], 1)
        return {'type': 'MaxPooling', 'strides': strides}
    elif layer_type == 'Dropout':
        rate = dropout_options.get(encoded_params[1], 0.2)
        return {'type': 'Dropout', 'rate': rate}
    elif layer_type == 'Dense':
        units = max(1, min(encoded_params[1], 512))  # Limitar unidades entre 1 y 512
        activation = activation_options.get(encoded_params[2], 'relu')
        return {'type': 'Dense', 'units': units, 'activation': activation}
    elif layer_type == 'Flatten':
        return {'type': 'Flatten'}
    elif layer_type == 'Repetition':
        return {
            'type': 'Repetition',
            'repetition_layers': int(encoded_params[1]),
            'repetition_count': int(encoded_params[2])
        }
    elif layer_type == 'DontCare':
        return {'type': "DontCare"}

    return None

# Ejemplos de codificación y decodificación
encoded_conv2d = encode_layer_params(0, 16, 0, 0)  # Conv2D con 16 filtros, stride 1 y activación ReLU
decoded_conv2d = decode_layer_params(encoded_conv2d)
print(f"\nCodificación real de Conv2D: {encoded_conv2d}")
print(f"Decodificación Conv2D: {decoded_conv2d}")

encoded_dropout = encode_layer_params(3, 1)  # Dropout con tasa de 0.3
decoded_dropout = decode_layer_params(encoded_dropout)
print(f"\nCodificación real de Dropout: {encoded_dropout}")
print(f"Decodificación Dropout: {decoded_dropout}")

encoded_dense = encode_layer_params(4, 128, 0)  # Dense con 128 neuronas y activación ReLU
decoded_dense = decode_layer_params(encoded_dense)
print(f"\nCodificación real de Dense: {encoded_dense}")
print(f"Decodificación Dense: {decoded_dense}")

encoded_repetition = encode_layer_params(8, 3, 5)  # Repetition para repetir las últimas 3 capas 5 veces
decoded_repetition = decode_layer_params(encoded_repetition)
print(f"\nCodificación real de Repetition: {encoded_repetition}")
print(f"Decodificación Repetition: {decoded_repetition}")



Codificación real de Conv2D: [0, 16, 0, 0]
Decodificación Conv2D: {'type': 'Conv2D', 'filters': 16, 'strides': 1, 'activation': 'relu'}

Codificación real de Dropout: [3, 1, 0, 0]
Decodificación Dropout: {'type': 'Dropout', 'rate': 0.3}

Codificación real de Dense: [4, 128, 0, 0]
Decodificación Dense: {'type': 'Dense', 'units': 128, 'activation': 'relu'}

Codificación real de Repetition: [8, 3, 5, 0]
Decodificación Repetition: {'type': 'Repetition', 'repetition_layers': 3, 'repetition_count': 5}


## Complete archs


In [3]:
# Clase para capas neutrales 'DontCare'
class DontCareLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(DontCareLayer, self).__init__()

    def call(self, inputs):
        return inputs

In [4]:
def encode_model_architecture(model_dict, max_alleles=48):
    """
    Codifica la arquitectura del modelo en una lista de valores con un máximo de `max_alleles`.
    Cada capa se codifica en función de sus parámetros.
    """
    encoded_layers = []
    total_alleles = 0

    for layer in model_dict['layers']:
        if layer['type'] == 'Repetition':  # Codificar capa de repetición
            encoded_layer = encode_layer_params(
                layer_type_idx=8,  # índice para 'Repetition'
                param1=layer.get('repetition_layers', 0),
                param2=layer.get('repetition_count', 1)
            )
        else:
            layer_type_idx = next(
                key for key, value in layer_type_options.items() if value == layer['type']
            )
            
            # Codificar parámetros específicos de cada tipo de capa
            if layer['type'] in ['Conv2D', 'DepthwiseConv2D']:  
                # Limitar filtros dentro del rango [4, 32]
                param1 = max(4, min(layer.get('filters', 8), 32))  
                param2 = next((key for key, value in stride_options.items() if value == layer.get('strides', 1.0)), 0)
                param3 = next((key for key, value in activation_options.items() if value == layer.get('activation', 'relu')), 0)
                encoded_layer = [layer_type_idx, param1, param2, param3]

            elif layer['type'] == 'Dense':
                # Limitar neuronas dentro del rango [1, 512]
                param1 = max(1, min(layer.get('units', 1), 512))
                param2 = next((key for key, value in activation_options.items() if value == layer.get('activation', 'relu')), 0)
                encoded_layer = [layer_type_idx, param1, param2, 0]

            elif layer['type'] == 'MaxPooling':
                param1 = next((key for key, value in stride_options.items() if value == layer.get('strides', 1.0)), 0)
                encoded_layer = [layer_type_idx, param1, 0, 0]

            elif layer['type'] == 'Dropout':
                param1 = next((key for key, value in dropout_options.items() if value == layer.get('rate', 0.2)), 0)
                encoded_layer = [layer_type_idx, param1, 0, 0]

            elif layer['type'] == 'BatchNorm':
                encoded_layer = [layer_type_idx, 0, 0, 0]

            elif layer['type'] == 'Flatten':
                encoded_layer = [layer_type_idx, 0, 0, 0]

            elif layer['type'] == 'DontCare':
                encoded_layer = [layer_type_idx, 0, 0, 0]

        # Añadir la codificación de la capa a la lista de alelos
        encoded_layers.extend(encoded_layer)
        total_alleles += len(encoded_layer)

    # Rellenar con 'DontCare' si el total de alelos es menor que `max_alleles`
    while total_alleles < max_alleles:
        dont_care_encoding = encode_layer_params(7)  # índice de 'DontCare'
        encoded_layers.extend(dont_care_encoding)
        total_alleles += len(dont_care_encoding)

    # Recortar si excede `max_alleles`
    final_encoding = encoded_layers[:max_alleles]
    print(f"Final Encoded Model: {final_encoding}")
    
    return final_encoding


In [5]:
import random

def fixArch(encoded_model, verbose=False):
    """
    Corrige la arquitectura codificada del modelo, asegurando que:
    - Se evite la presencia de capas incompatibles después de una capa Flatten.
    - En caso de una capa de Repetition, se ajuste el alcance de repetición si no hay suficientes capas anteriores.
    
    Parameters:
        encoded_model (list): Lista codificada de la arquitectura del modelo.
        verbose (bool): Si es True, muestra las correcciones realizadas.

    Returns:
        list: Lista con la arquitectura corregida, truncada a un máximo de 48 alelos.
    """
    
    fixed_layers = []  # Lista que almacenará la arquitectura corregida
    input_is_flattened = False  # Indicador para saber si ya hay una capa Flatten en el modelo
    index = 0  # Índice para recorrer el modelo codificado

    # Procesar cada capa en el modelo sin forzar la primera capa a ser específica
    while index < len(encoded_model) and len(fixed_layers) < 48:
        layer_type = int(encoded_model[index])  # Obtener el tipo de capa actual

        # Procesar la capa de Repetition
        if layer_type == 8:
            repetition_layers = int(encoded_model[index + 1])  # Número de capas a repetir
            repetition_count = int(encoded_model[index + 2])  # Cantidad de repeticiones
            
            # Verificar si hay suficientes capas para la repetición solicitada
            actual_layers_to_repeat = min(repetition_layers, len(fixed_layers) // 4)
            
            if actual_layers_to_repeat != repetition_layers:
                if verbose:
                    print(f"Ajustando alcance de repetición de {repetition_layers} a {actual_layers_to_repeat} debido a falta de capas.")
                repetition_layers = actual_layers_to_repeat

            # Añadir la capa de repetición sin modificar su estructura
            fixed_layers.extend([layer_type, repetition_layers, repetition_count, 0])
            index += 4
            continue

        # Procesar cada tipo de capa normal con sus restricciones
        if layer_type == 0:  # Conv2D
            if input_is_flattened:
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                # Limitar el número de filtros entre 4 y 32
                filters = min(max(int(encoded_model[index + 1]), 4), 32)
                stride_idx = min(max(int(encoded_model[index + 2]), 0), 1)
                activation_idx = min(max(int(encoded_model[index + 3]), 0), 3)
                fixed_layers.extend([layer_type, filters, stride_idx, activation_idx])
        
        elif layer_type == 6:  # DepthwiseConv2D
            if input_is_flattened:
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                # Limitar el número de filtros entre 4 y 32
                filters = min(max(int(encoded_model[index + 1]), 4), 32)
                stride_idx = min(max(int(encoded_model[index + 2]), 0), 1)
                activation_idx = min(max(int(encoded_model[index + 3]), 0), 3)
                fixed_layers.extend([layer_type, filters, stride_idx, activation_idx])
        
        elif layer_type == 2:  # MaxPooling
            if input_is_flattened:
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                stride_idx = min(max(int(encoded_model[index + 1]), 0), 1)
                fixed_layers.extend([layer_type, stride_idx, 0, 0])
        
        elif layer_type == 3:  # Dropout
            rate_idx = min(max(int(encoded_model[index + 1]), 0), 3)
            fixed_layers.extend([layer_type, rate_idx, 0, 0])
        
        elif layer_type == 4:  # Dense
            # Limitar el número de neuronas entre 1 y 512
            neurons = min(max(int(encoded_model[index + 1]), 1), 512)
            activation_idx = min(max(int(encoded_model[index + 2]), 0), 3)
            fixed_layers.extend([layer_type, neurons, activation_idx, 0])
        
        elif layer_type == 1:  # BatchNorm
            fixed_layers.extend([layer_type, 0, 0, 0])
        
        elif layer_type == 5:  # Flatten
            if len(fixed_layers) < 16:
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            elif input_is_flattened:
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                fixed_layers.extend([layer_type, 0, 0, 0])
                input_is_flattened = True  # Marcar que ya hay un Flatten
        
        elif layer_type == 7:  # DontCare
            fixed_layers.extend([layer_type, 0, 0, 0])

        index += 4  # Avanzar al siguiente grupo de parámetros

    return fixed_layers[:48]  # Limitar a 48 alelos


In [6]:
# Ejecutar ejemplos de prueba para la función fixArch con arquitecturas de 12 capas (48 valores en total)

# Ejemplo 1: Arquitectura sin Flatten antes de Dense y con capas adicionales
encoded_model_1 = [
    0, 3, 0, 0,    # Conv2D, 32 filtros, stride 1, activación relu
    2, 1, 0, 0,    # MaxPooling, stride 2
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu (sin Flatten antes)
    7, 0, 0, 0,    # DontCare
    1, 0, 0, 0,    # BatchNorm
    3, 2, 0, 0,    # Dropout, tasa 0.4
    0, 1, 1, 0,    # Conv2D, 16 filtros, stride 2, activación relu
    6, 1, 1, 0,    # DepthwiseConv2D, 16 filtros, stride 2, activación relu
    5, 0, 0, 0,    # Flatten
    4, 3, 2, 0,    # Dense, 256 neuronas, activación sigmoid
    4, 1, 3, 0,    # Dense, 32 neuronas, activación tanh
    4, 1, 2, 0     # Dense, 32 neuronas, activación sigmoid
]

# Ejemplo 2: Arquitectura con Flatten al inicio (inválido) y sin Dense al final
encoded_model_2 = [
    5, 0, 0, 0,    # Flatten (inválido al inicio)
    0, 2, 1, 1,    # Conv2D, 30 filtros, stride 2, activación leaky_relu
    2, 1, 0, 0,    # MaxPooling, stride 2
    3, 1, 0, 0,    # Dropout, tasa 0.3
    1, 0, 0, 0,    # BatchNorm
    0, 1, 0, 0,    # Conv2D, 16 filtros, stride 1, activación relu
    6, 2, 1, 0,    # DepthwiseConv2D, 30 filtros, stride 2, activación relu
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu
    3, 0, 0, 0,    # Dropout, tasa 0.2
    1, 0, 0, 0,    # BatchNorm
    4, 3, 3, 0,    # Dense, 256 neuronas, activación tanh
    7, 0, 0, 0     # DontCare
]

# Ejemplo 3: Arquitectura con Repetition y sin Dense al final
encoded_model_3 = [
    0, 1, 0, 0,    # Conv2D, 16 filtros, stride 1, activación relu
    8, 2, 2, 0,    # Repetition, repite las últimas 2 capas 2 veces
    5, 0, 0, 0,    # Flatten
    2, 1, 0, 0,    # MaxPooling, stride 2
    3, 1, 0, 0,    # Dropout, tasa 0.3
    6, 2, 1, 0,    # DepthwiseConv2D, 30 filtros, stride 2, activación relu
    0, 3, 0, 1,    # Conv2D, 32 filtros, stride 1, activación leaky_relu
    1, 0, 0, 0,    # BatchNorm
    5, 0, 0, 0,    # Flatten
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu
    7, 0, 0, 0,    # DontCare
    3, 2, 0, 0     # Dropout, tasa 0.4
]

# Ejemplo 4: Arquitectura con parámetros fuera de límite
encoded_model_4 = [
    0, 4, 2, 1,    # Conv2D, 64 filtros (fuera de límite), stride 2, activación leaky_relu
    3, 5, 0, 0,    # Dropout, tasa 0.6 (fuera de límite)
    4, 5, 3, 0,    # Dense, 512 neuronas (fuera de límite), activación sigmoid
    6, 4, 2, 0,    # DepthwiseConv2D, 64 filtros (fuera de límite), stride 2, activación relu
    1, 0, 0, 0,    # BatchNorm
    3, 3, 0, 0,    # Dropout, tasa 0.5
    5, 0, 0, 0,    # Flatten
    2, 1, 0, 0,    # MaxPooling, stride 2
    0, 1, 0, 0,    # Conv2D, 16 filtros, stride 1, activación relu
    4, 3, 1, 0,    # Dense, 256 neuronas, activación leaky_relu
    7, 0, 0, 0,    # DontCare
    4, 1, 2, 0     # Dense, 32 neuronas, activación sigmoid
]
# Ejemplo 5: Arquitectura con capa de Repetition que repite las últimas capas
encoded_model_5 = [
    0, 2, 1, 0,    # Conv2D, 30 filtros, stride 2, activación relu
    8, 2, 3, 0,    # Repetition, repite las últimas 2 capas 3 veces
    5, 0, 0, 0,    # Flatten
    2, 1, 0, 0,    # MaxPooling, stride 2
    3, 1, 0, 0,    # Dropout, tasa 0.3
    6, 2, 1, 0,    # DepthwiseConv2D, 30 filtros, stride 2, activación relu
    0, 3, 0, 1,    # Conv2D, 32 filtros, stride 1, activación leaky_relu
    1, 0, 0, 0,    # BatchNorm
    5, 0, 0, 0,    # Flatten
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu
    7, 0, 0, 0,    # DontCare
    3, 2, 0, 0     # Dropout, tasa 0.4
]

# Lista de modelos para probar
model_examples = [encoded_model_1, encoded_model_2, encoded_model_3, encoded_model_4, encoded_model_5]

# Ejecución de pruebas
for i, model in enumerate(model_examples, 1):
    print(f"\n--- Ejemplo {i} ---")
    fixed_model = fixArch(model, verbose=True)
    print(f"Modelo Original: {len(model)} alelos\n{model}") 
    print(f"Modelo Corregido: {len(fixed_model)} alelos\n{fixed_model}")



--- Ejemplo 1 ---
Modelo Original: 48 alelos
[0, 3, 0, 0, 2, 1, 0, 0, 4, 2, 1, 0, 7, 0, 0, 0, 1, 0, 0, 0, 3, 2, 0, 0, 0, 1, 1, 0, 6, 1, 1, 0, 5, 0, 0, 0, 4, 3, 2, 0, 4, 1, 3, 0, 4, 1, 2, 0]
Modelo Corregido: 48 alelos
[0, 4, 0, 0, 2, 1, 0, 0, 4, 2, 1, 0, 7, 0, 0, 0, 1, 0, 0, 0, 3, 2, 0, 0, 0, 4, 1, 0, 6, 4, 1, 0, 5, 0, 0, 0, 4, 3, 2, 0, 4, 1, 3, 0, 4, 1, 2, 0]

--- Ejemplo 2 ---
Modelo Original: 48 alelos
[5, 0, 0, 0, 0, 2, 1, 1, 2, 1, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 6, 2, 1, 0, 4, 2, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 4, 3, 3, 0, 7, 0, 0, 0]
Modelo Corregido: 48 alelos
[7, 0, 0, 0, 0, 4, 1, 1, 2, 1, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 6, 4, 1, 0, 4, 2, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 4, 3, 3, 0, 7, 0, 0, 0]

--- Ejemplo 3 ---
Ajustando alcance de repetición de 2 a 1 debido a falta de capas.
Modelo Original: 48 alelos
[0, 1, 0, 0, 8, 2, 2, 0, 5, 0, 0, 0, 2, 1, 0, 0, 3, 1, 0, 0, 6, 2, 1, 0, 0, 3, 0, 1, 1, 0, 0, 0, 5, 0, 0, 0, 4, 2, 1, 0, 7, 0, 0, 0, 3, 2, 0, 0]
Modelo Corregid

In [7]:
def decode_model_architecture(encoded_model):
    """
    Decodifica la arquitectura del modelo a partir de la lista codificada de valores (índices),
    aplicando las reglas de repetición y asegurando la inclusión de una capa convolucional inicial.
    """
    model_dict = {'layers': [{'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}]}  # Inserta Conv2D inicial
    index = 0

    while index < len(encoded_model):
        layer_type = int(encoded_model[index])
        param1 = encoded_model[index + 1]
        param2 = encoded_model[index + 2]
        param3 = encoded_model[index + 3]

        if layer_type == 8:  # Capa de Repetition
            repetition_layers = int(param1)
            repetition_count = int(param2)
            # Selecciona solo el grupo válido de capas para la repetición
            layers_to_repeat = select_group_for_repetition(model_dict['layers'], repetition_layers)
            
            if len(layers_to_repeat) > 0:
                for _ in range(repetition_count):
                    model_dict['layers'].extend(layers_to_repeat)

        else:
            decoded_layer = {}

            if layer_type == 0:  # Conv2D
                decoded_layer = {
                    'type': 'Conv2D',
                    'filters': max(4, min(param1, 32)),  # Limita `filters` entre 4 y 32
                    'strides': stride_options.get(param2, 1),
                    'activation': activation_options.get(param3, 'relu')
                }
            elif layer_type == 6:  # DepthwiseConv2D
                decoded_layer = {
                    'type': 'DepthwiseConv2D',
                    'filters': max(4, min(param1, 32)),  # Limita `filters` entre 4 y 32
                    'strides': stride_options.get(param2, 1),
                    'activation': activation_options.get(param3, 'relu')
                }
            elif layer_type == 2:  # MaxPooling
                decoded_layer = {
                    'type': 'MaxPooling',
                    'strides': stride_options.get(param1, 1)
                }
            elif layer_type == 3:  # Dropout
                decoded_layer = {
                    'type': 'Dropout',
                    'rate': dropout_options.get(param1, 0.2)
                }
            elif layer_type == 4:  # Dense
                decoded_layer = {
                    'type': 'Dense',
                    'units': max(1, min(param1, 512)),  # Limita `units` entre 1 y 512
                    'activation': activation_options.get(param2, 'relu')
                }
            elif layer_type == 1:  # BatchNorm
                decoded_layer = {'type': 'BatchNorm'}
            elif layer_type == 5:  # Flatten
                decoded_layer = {'type': 'Flatten'}
            elif layer_type == 7:  # DontCare
                decoded_layer = {'type': 'DontCare'}

            model_dict['layers'].append(decoded_layer)

        index += 4

    # Asegura que haya una capa Flatten antes de la capa Dense final, si no ya existe una Flatten
    if model_dict['layers'][-1]['type'] != 'Flatten':
        model_dict['layers'].append({'type': 'Flatten'})
        
    # Añade la capa Dense final obligatoria
    model_dict['layers'].append({'type': 'Dense', 'units': 1, 'activation': 'sigmoid'})

    return model_dict

def select_group_for_repetition(layers, repetition_layers):
    """
    Selecciona el primer grupo válido para repetición en función de las reglas de compatibilidad.

    Parameters:
        layers (list): Lista de capas ya procesadas, donde cada capa es un diccionario.
        repetition_layers (int): Número de capas hacia atrás para considerar en la repetición.

    Returns:
        list: Lista de capas compatibles para repetición.
    """
    valid_layers = []
    group_type = None

    # Retrocede desde el final de `layers` para encontrar el grupo válido
    for layer in reversed(layers[-repetition_layers:]):
        if group_type is None:
            # Determina el tipo de grupo
            if layer['type'] in ['Flatten', 'Dense']:
                group_type = 'dense'
                valid_layers.insert(0, layer)
            elif layer['type'] in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling']:
                group_type = 'convolutional'
                valid_layers.insert(0, layer)
            elif layer['type'] in ['BatchNorm', 'DontCare']:  # BatchNorm y DontCare son compatibles con ambos grupos
                valid_layers.insert(0, layer)
        else:
            # Agrega solo capas compatibles con el grupo seleccionado
            if group_type == 'dense' and layer['type'] in ['Flatten', 'Dense', 'BatchNorm', 'DontCare']:
                valid_layers.insert(0, layer)
            elif group_type == 'convolutional' and layer['type'] in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling', 'BatchNorm', 'DontCare']:
                valid_layers.insert(0, layer)

    return valid_layers


In [8]:
# Reparar cada ejemplo con fixArch y luego decodificar con decode_model_architecture
repaired_and_decoded_models = []

for i, encoded_model in enumerate([encoded_model_1, encoded_model_2, encoded_model_3, encoded_model_4, encoded_model_5], 1):
    # Reparar el modelo
    repaired_model = fixArch(encoded_model, verbose=True)
    # Decodificar el modelo reparado
    decoded_model = decode_model_architecture(repaired_model)
    # Guardar el modelo decodificado
    repaired_and_decoded_models.append(decoded_model)
    print(f"\n--- Modelo {i} Decodificado ---")
    print(decoded_model)

# Opcional: Mostrar todos los modelos decodificados juntos
print("\n--- Todos los Modelos Decodificados ---")
for i, model in enumerate(repaired_and_decoded_models, 1):
    print(f"Modelo {i} Decodificado:", model)


--- Modelo 1 Decodificado ---
{'layers': [{'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'Conv2D', 'filters': 4, 'strides': 1, 'activation': 'relu'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Dense', 'units': 2, 'activation': 'leaky_relu'}, {'type': 'DontCare'}, {'type': 'BatchNorm'}, {'type': 'Dropout', 'rate': 0.4}, {'type': 'Conv2D', 'filters': 4, 'strides': 2, 'activation': 'relu'}, {'type': 'DepthwiseConv2D', 'filters': 4, 'strides': 2, 'activation': 'relu'}, {'type': 'Flatten'}, {'type': 'Dense', 'units': 3, 'activation': 'sigmoid'}, {'type': 'Dense', 'units': 1, 'activation': 'tanh'}, {'type': 'Dense', 'units': 1, 'activation': 'sigmoid'}, {'type': 'Flatten'}, {'type': 'Dense', 'units': 1, 'activation': 'sigmoid'}]}

--- Modelo 2 Decodificado ---
{'layers': [{'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'DontCare'}, {'type': 'Conv2D', 'filters': 4, 'strides': 2, 'activation': 'leaky_relu'}, {'type': 'MaxPo

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization, DepthwiseConv2D



def build_tf_model_from_dict(model_dict, input_shape=(28, 28, 3)):
    """
    Construye un modelo de TensorFlow a partir de un diccionario JSON expandido.
    """
    print("\nConstruyendo el modelo en TensorFlow desde el JSON expandido...")
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=input_shape))

    for layer in model_dict['layers']:
        if layer['type'] == 'Conv2D':
            model.add(Conv2D(filters=layer['filters'], kernel_size=(3, 3), strides=int(layer['strides']), padding='same', activation=layer['activation']))
        
        elif layer['type'] == 'DepthwiseConv2D':
            model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=int(layer['strides']), padding='same', activation=layer['activation']))
        
        elif layer['type'] == 'BatchNorm':
            model.add(BatchNormalization())
        
        elif layer['type'] == 'MaxPooling':
            model.add(MaxPooling2D(pool_size=(2, 2), strides=int(layer['strides']), padding='same'))
        
        elif layer['type'] == 'Flatten':
            model.add(Flatten())
        
        elif layer['type'] == 'Dense':
            model.add(Dense(units=int(layer['units']), activation=layer['activation']))
        
        elif layer['type'] == 'Dropout':
            model.add(Dropout(rate=layer['rate']))
        
        elif layer['type'] == 'DontCare':
            model.add(DontCareLayer())
    
    return model


In [10]:
# Ejemplo 1: build_CNN_LF_model
model_CNN_LF = {
    "layers": [
        {"type": "Conv2D", "filters": 30, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},  # Revisar 'filters'
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.3},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 2: build_reduced_model
model_reduced = {
    "layers": [
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "leaky_relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 3: build_Spectro_CNN_model
model_spectro_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "MaxPooling", "strides": 2}, 
         
         
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
    {"type":"Flatten"},
    {"type":"Dense","units":256,"activation":"relu"},   
    {"type":"Dropout","rate":0.5},
                
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo corregido: build_Spectro_CNN_model con capas de repetición
model_spectro_CNN_with_repetition = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2}, 
        # Aquí indicamos que las siguientes 10 capas (5 Conv2D + 5 BatchNorm) se repiten 10 veces
        {"type": "Repetition", "repetition_layers": 3, "repetition_count": 31},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}


# Ejemplo 4: Simple Conv2D model
model_simple_conv = {
    "layers": [
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 5: Simple Dense model
model_dense_only = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 256, "activation": "sigmoid"}
    ]
}

# Ejemplo 6: Small CNN model with Dropout
model_small_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 7: Deep CNN model
model_deep_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 10, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 8: Basic Dense with Dropout
model_dense_dropout = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 32, "activation": "tanh"}
    ]
}

model_with_depthwise = {
    "layers": [
        {"type": "DepthwiseConv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Dense", "units": 128, "activation": "sigmoid"},
        {"type": "Dense", "units": 1, "activation": "tanh"}
    ]
}



In [11]:
def process_model_pipeline(model_dict, input_shape=(28, 28, 3), verbose=False):
    """
    Procesa el pipeline de creación del modelo desde el JSON inicial hasta el modelo de TensorFlow.
    
    Parameters:
        model_dict (dict): Modelo en formato JSON.
        input_shape (tuple): Forma de entrada para el modelo.
        verbose (bool): Si es True, muestra los detalles de cada etapa del proceso.
        
    Returns:
        tf.keras.Model: El modelo de TensorFlow construido a partir del JSON procesado.
    """
    # Codificar el modelo
    encoded_model = encode_model_architecture(model_dict, max_alleles=48)
    if verbose:
        print(f"\nModelo codificado: {encoded_model}")

    # Reparar el modelo codificado
    fixed_model = fixArch(encoded_model, verbose=verbose)
    if verbose:
        print(f"\nModelo codificado y reparado: {fixed_model}")

    # Decodificar el modelo reparado
    decoded_model = decode_model_architecture(fixed_model)
    if verbose:
        print(f"\nModelo decodificado:\n{decoded_model}")

    # Construir el modelo de TensorFlow
    tf_model = build_tf_model_from_dict(decoded_model, input_shape=input_shape)
    if verbose:
        print("\nModelo de TensorFlow construido.")
        tf_model.summary()
    
    return tf_model


# Ejemplo de uso con un modelo y verbose activado



In [12]:

# Verificación de los modelos
print("Verifications:")

print("\nModel 1: CNN_LF")
tf_model_example = process_model_pipeline(model_CNN_LF, verbose=True)  # Ejemplo 1

print("\nModel 2: Reduced")
tf_model_example = process_model_pipeline(model_reduced, verbose=True)  # Ejemplo 2

print("\nModel 3: Spectro CNN")
tf_model_example = process_model_pipeline(model_spectro_CNN, verbose=True)  # Ejemplo 3

print("\nModel 3.5 Spectro CNN with repetition")
tf_model_example = process_model_pipeline(model_spectro_CNN_with_repetition, verbose=True)  # Ejemplo 3

print("\nModel 4: Simple Conv2D")
tf_model_example = process_model_pipeline(model_simple_conv, verbose=True)  # Ejemplo 4

print("\nModel 5: Dense Only")
tf_model_example = process_model_pipeline(model_dense_only, verbose=True)  # Ejemplo 5

print("\nModel 6: Small CNN")
tf_model_example = process_model_pipeline(model_small_CNN, verbose=True)  # Ejemplo 6

print("\nModel 7: Deep CNN")
tf_model_example = process_model_pipeline(model_deep_CNN, verbose=True)  # Ejemplo 7

print("\nModel 8: Dense with Dropout")
tf_model_example = process_model_pipeline(model_dense_dropout, verbose=True)  # Ejemplo 8

print("\nModel with DepthwiseConv2D")
tf_model_example = process_model_pipeline(model_with_depthwise)



Verifications:

Model 1: CNN_LF
Final Encoded Model: [0, 30, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 16, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 5, 0, 0, 0, 4, 256, 0, 0, 3, 1, 0, 0, 4, 1, 2, 0]

Modelo codificado: [0, 30, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 16, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 5, 0, 0, 0, 4, 256, 0, 0, 3, 1, 0, 0, 4, 1, 2, 0]

Modelo codificado y reparado: [0, 30, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 16, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 5, 0, 0, 0, 4, 256, 0, 0, 3, 1, 0, 0, 4, 1, 2, 0]

Modelo decodificado:
{'layers': [{'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'Conv2D', 'filters': 30, 'strides': 1, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.2}, {'type': 'BatchNorm'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Conv2D', 'filters': 16, 'strides': 1, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.2}, {'type': 'BatchNorm'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Fl

# Testing random generated architectures


In [13]:
import random
import tensorflow as tf

def generate_random_architecture():
    num_layers = 12
    layers = []
   
    for _ in range(num_layers):
        layer_type = random.choice([
            'Conv2D', 'DepthwiseConv2D', 'BatchNorm', 'MaxPooling', 'Dropout', 
            'Dense', 'Flatten', 'DontCare', 'Repetition'
        ])
        layers.append(generate_layer(layer_type))
    return {"layers": layers}

def generate_layer(layer_type):
    if layer_type == 'Conv2D':
        return {
            "type": "Conv2D",
            "filters": random.randint(4, 16),  # Rango reducido para filtros
            "strides": random.choice([1, 2]),
            "activation": random.choice(["relu", "leaky_relu", "sigmoid", "tanh"])
        }
    elif layer_type == 'DepthwiseConv2D':
        return {
            "type": "DepthwiseConv2D",
            "filters": random.randint(4, 16),  # Rango reducido para filtros
            "strides": random.choice([1, 2]),
            "activation": random.choice(["relu", "leaky_relu", "sigmoid", "tanh"])
        }
    elif layer_type == 'BatchNorm':
        return {"type": "BatchNorm"}
    elif layer_type == 'MaxPooling':
        return {
            "type": "MaxPooling",
            "strides": random.choice([1, 2])
        }
    elif layer_type == 'Dropout':
        return {
            "type": "Dropout",
            "rate": random.choice([0.2, 0.3, 0.4, 0.5])
        }
    elif layer_type == 'Dense':
        return {
            "type": "Dense",
            "units": random.randint(1, 128),  # Rango reducido para unidades
            "activation": random.choice(["relu", "leaky_relu", "sigmoid", "tanh"])
        }
    elif layer_type == 'Flatten':
        return {"type": "Flatten"}
    elif layer_type == 'DontCare':
        return {"type": "DontCare"}
    elif layer_type == 'Repetition':
        return {
            "type": "Repetition",
            "repetition_layers": random.randint(1, 3),  # Limitar el número de capas para repetir
            "repetition_count": random.randint(1, 2)  # Limitar el conteo de repeticiones
        }
    return {}

def generate_and_compare_repaired_models(num_models=1000):
    repaired_models = set()
    duplicate_count = 0

    for i in range(num_models):
        random_architecture = generate_random_architecture()
        repaired_architecture = process_model_pipeline(random_architecture)
        repaired_architecture_str = str(repaired_architecture)

        if repaired_architecture_str in repaired_models:
            duplicate_count += 1
        else:
            repaired_models.add(repaired_architecture_str)
        
        # Liberar memoria
        tf.keras.backend.clear_session()

    unique_count = len(repaired_models)
    print(f"\nModelos únicos después de la reparación: {unique_count}")
    print(f"Modelos duplicados después de la reparación: {duplicate_count}")

generate_and_compare_repaired_models(num_models=200)


Final Encoded Model: [2, 1, 0, 0, 0, 13, 1, 2, 2, 1, 0, 0, 7, 0, 0, 0, 2, 1, 0, 0, 8, 2, 1, 0, 4, 13, 1, 0, 0, 7, 1, 0, 7, 0, 0, 0, 1, 0, 0, 0, 4, 86, 0, 0, 0, 8, 1, 2]

Construyendo el modelo en TensorFlow desde el JSON expandido...
Final Encoded Model: [8, 3, 2, 0, 8, 2, 1, 0, 2, 0, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 4, 118, 0, 0, 5, 0, 0, 0, 4, 19, 1, 0, 7, 0, 0, 0, 7, 0, 0, 0, 6, 5, 0, 0]

Construyendo el modelo en TensorFlow desde el JSON expandido...
Final Encoded Model: [5, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 4, 127, 1, 0, 6, 7, 1, 2, 3, 0, 0, 0, 5, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0]

Construyendo el modelo en TensorFlow desde el JSON expandido...
Final Encoded Model: [8, 2, 1, 0, 6, 10, 0, 2, 5, 0, 0, 0, 8, 1, 2, 0, 4, 27, 1, 0, 4, 27, 2, 0, 4, 5, 3, 0, 8, 2, 2, 0, 3, 1, 0, 0, 5, 0, 0, 0, 8, 2, 1, 0, 8, 1, 2, 0]

Construyendo el modelo en TensorFlow desde el JSON expandido...
Final Encoded Model: [6, 13, 1, 0, 3, 0, 0, 0, 6, 11, 1, 2, 0, 9,

# Entrenando el modelo surogado


In [17]:
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torchaudio
import torch
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Resizing, Conv2D, Dropout, BatchNormalization, MaxPooling2D, MaxPool2D, Flatten, Dense, Input, LeakyReLU
from tqdm import tqdm
import tensorflow_addons as tfa

# Importar funciones previamente definidas para codificar, decodificar y reparar arquitecturas
# Y otras dependencias específicas como `build_tf_model_from_dict`, `generate_random_architecture`, etc.

# Configuración de parámetros
class Config:
    def __init__(self, architecture='random', epochs=50, sample_rate=None, time=5, n_splits=5, window_size=5):
        self.architecture = architecture
        self.epochs = epochs
        self.sample_rate = sample_rate
        self.time = time
        self.n_splits = n_splits
        self.window_size = window_size

# Cargar datos de audio
def load_audio_data(directory, window_size, sample_rate):
    audio_dict = {}
    for file_name in os.listdir(directory):
        if file_name.endswith(".wav"):
            waveform, sr = torchaudio.load(os.path.join(directory, file_name))
            if sample_rate is None:
                sample_rate = sr
            num_windows = int(waveform.shape[1] / (window_size * sample_rate))
            for i in range(num_windows):
                start = i * window_size * sample_rate
                end = (i + 1) * window_size * sample_rate
                audio_dict[f"{file_name}_{i}"] = waveform[:, start:end].numpy()
    return audio_dict, sample_rate

# Preprocesar datos de audio
def preprocess_audio(audio_dict, sample_rate):
    audio_dict = copy.deepcopy(audio_dict)
    n_mels = 128
    n_fft = int(sample_rate * 0.029)
    hop_length = int(sample_rate * 0.010)
    win_length = int(sample_rate * 0.025)

    for filename, waveform in tqdm(audio_dict.items(), desc='MELSPECTROGRAM'):
        waveform = torch.from_numpy(waveform)
        spec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_fft=n_fft, n_mels=n_mels, hop_length=hop_length, win_length=win_length)(waveform)
        spec = torchaudio.transforms.AmplitudeToDB()(spec)
        spec = spec.numpy()
        spec = (spec - spec.min()) / (spec.max() - spec.min())
        audio_dict[filename] = spec
    return audio_dict

# Padding de los espectrogramas
def pad_and_crop_spectrograms(spectrograms, target_shape=(128, 128)):
    padded_spectrograms = []
    for spec in spectrograms:
        if spec.shape[0] > target_shape[0]:
            spec = spec[:target_shape[0], :]
        if spec.shape[1] > target_shape[1]:
            spec = spec[:, :target_shape[1]]
        
        pad_width = [(0, max(0, target_shape[0] - spec.shape[0])), 
                     (0, max(0, target_shape[1] - spec.shape[1]))]
        
        padded_spec = np.pad(spec, pad_width, mode='constant')
        padded_spectrograms.append(padded_spec)
    return np.array(padded_spectrograms)

# Split de audio en train y test
def train_test_split_audio(audio_dict):
    df = pd.read_csv('Dataset.csv', usecols=['Participant_ID', 'PHQ-9 Score'], dtype={1: str})
    df['labels'] = np.zeros([len(df),], dtype=int)
    df.loc[df['PHQ-9 Score'] < 10, 'labels'] = 0
    df.loc[df['PHQ-9 Score'] >= 10, 'labels'] = 1

    labels = df.set_index('Participant_ID').to_dict()['labels']

    X, Y = [], []
    for filename, data in tqdm(audio_dict.items(), 'LABEL'):
        ID = filename[:3]
        if ID in labels:
            dep = 0 if labels[ID] == 0 else 1
            [X.append(x) for x in data]
            [Y.append(dep) for x in data]

    X = pad_and_crop_spectrograms(X)
    Y = np.array(Y)

    X = X[..., np.newaxis]
    print(f"X shape: {X.shape}, Y shape: {Y.shape}")
    return X, Y


# Función de especificidad
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Entrenamiento y evaluación de cada arquitectura decodificada
def train_and_evaluate_model(model, X_train, Y_train, X_val, Y_val, X_test, Y_test, config):
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=["accuracy", 'Precision', 'Recall'])
    model.fit(X_train, Y_train, epochs=config.epochs, validation_data=(X_val, Y_val), verbose=0)
    results = model.evaluate(X_test, Y_test, verbose=0)

    # Obtener predicciones para métricas adicionales
    Y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = results[1]
    precision = precision_score(Y_test, Y_pred)
    recall = recall_score(Y_test, Y_pred)
    f1 = f1_score(Y_test, Y_pred)
    specificity = specificity_score(Y_test, Y_pred)

    return [results[0], accuracy, precision, recall, f1, specificity]

# Pipeline para generar, decodificar y entrenar modelos
def generate_and_train_models(num_models, directory='./SM-27', target_shape=(128, 128, 1), use_kfold=True):
    """
    Genera, entrena y evalúa múltiples modelos con arquitecturas aleatorias, 
    y guarda los resultados de cada uno en un archivo CSV.
    
    Parameters:
        num_models (int): Número de modelos a generar y entrenar.
        directory (str): Directorio con archivos de audio.
        target_shape (tuple): Forma objetivo de los espectrogramas.
        use_kfold (bool): Si es True, se usa K-Fold Cross Validation; si no, solo un split para entrenamiento y validación.
    """
    # Configuración inicial
    results_data = []
    config = Config(architecture='random', epochs=50)
    
    # Cargar y preprocesar datos de audio
    print("Cargando y preprocesando datos de audio...")
    audio_dict, sample_rate = load_audio_data(directory, config.window_size, config.sample_rate)
    audio_dict = preprocess_audio(audio_dict, sample_rate)
    X, Y = train_test_split_audio(audio_dict)
    X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    # Configuración de KFold para validación cruzada si se usa
    if use_kfold:
        kfold = KFold(n_splits=config.n_splits, shuffle=True)
    
    for i in range(num_models):
        print(f"\nProcesando modelo {i + 1}/{num_models}...")
        
        # Generar una arquitectura aleatoria y codificada
        print("Generando y codificando arquitectura aleatoria...")
        random_architecture = generate_random_architecture()
        encoded_architecture = encode_model_architecture(random_architecture)
        
        # Decodificar y reparar la arquitectura
        print("Reparando y decodificando la arquitectura...")
        repaired_architecture = fixArch(encoded_architecture)  # Reparar si es necesario
        decoded_model_dict = decode_model_architecture(repaired_architecture)
        
        # Almacenar la codificación y los resultados de evaluación
        model_results = [encoded_architecture]

        # Si `use_kfold` es True, se realiza K-Fold Cross Validation
        if use_kfold:
            fold_results = []
            for fold, (train_index, val_index) in enumerate(kfold.split(X_train_val)):
                print(f"Entrenando fold {fold + 1}/{config.n_splits} del modelo {i + 1}...")
                X_train, X_val = X_train_val[train_index], X_train_val[val_index]
                Y_train, Y_val = Y_train_val[train_index], Y_train_val[val_index]
                
                # Construir y entrenar el modelo
                tf_model = build_tf_model_from_dict(decoded_model_dict, input_shape=(target_shape[0], target_shape[1], 1))
                fold_results.append(train_and_evaluate_model(tf_model, X_train, Y_train, X_val, Y_val, X_test, Y_test, config))
                print(f"Fold {fold + 1} completado para modelo {i + 1}.")

            # Promediar los resultados de los pliegues
            avg_results = np.mean(fold_results, axis=0)
            model_results.extend(avg_results)
        
        else:
            # Entrenamiento sin K-Fold Cross Validation
            print(f"Entrenando modelo {i + 1} sin K-Fold Cross Validation...")
            X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=0.2, random_state=42)
            tf_model = build_tf_model_from_dict(decoded_model_dict, input_shape=(target_shape[0], target_shape[1], 1))
            single_run_results = train_and_evaluate_model(tf_model, X_train, Y_train, X_val, Y_val, X_test, Y_test, config)
            model_results.extend(single_run_results)
            print(f"Modelo {i + 1} completado y resultados almacenados.")

        results_data.append(model_results)
    
    # Guardar resultados en CSV
    columns = ["Encoded Architecture", "Loss", "Accuracy", "Precision", "Recall", "F1", "Specificity"]
    results_df = pd.DataFrame(results_data, columns=columns)
    results_df.to_csv("./model_results_200.csv", index=False)
    print("Resultados guardados en 'model_results.csv'")

# Ejecutar la generación y entrenamiento de modelos sin K-Fold Cross Validation
generate_and_train_models(num_models=250, target_shape=(128, 128, 1),  use_kfold=False)





37/37 [==============================] - 1s 21ms/step
Modelo 52 completado y resultados almacenados.

Procesando modelo 53/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [1, 0, 0, 0, 8, 1, 1, 0, 7, 0, 0, 0, 0, 4, 1, 2, 6, 12, 0, 3, 7, 0, 0, 0, 3, 0, 0, 0, 0, 14, 1, 3, 1, 0, 0, 0, 6, 8, 1, 1, 0, 12, 0, 0, 7, 0, 0, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 53 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow desde el JSON expandido...


c:\Users\herna\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


37/37 [==============================] - 0s 5ms/step
Modelo 53 completado y resultados almacenados.

Procesando modelo 54/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [7, 0, 0, 0, 8, 3, 1, 0, 0, 16, 0, 3, 1, 0, 0, 0, 5, 0, 0, 0, 0, 10, 0, 1, 4, 100, 1, 0, 8, 2, 2, 0, 7, 0, 0, 0, 0, 4, 1, 1, 0, 5, 0, 3, 0, 14, 0, 3]
Reparando y decodificando la arquitectura...
Entrenando modelo 54 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow desde el JSON expandido...
37/37 [==============================] - 0s 7ms/step
Modelo 54 completado y resultados almacenados.

Procesando modelo 55/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [0, 15, 1, 0, 7, 0, 0, 0, 2, 1, 0, 0, 6, 6, 1, 3, 5, 0, 0, 0, 3, 0, 0, 0, 8, 3, 2, 0, 5, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 2, 0, 0, 0, 5, 0, 0, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 55 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow desde e

c:\Users\herna\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


37/37 [==============================] - 0s 4ms/step
Modelo 58 completado y resultados almacenados.

Procesando modelo 59/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [5, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 7, 0, 0, 0, 4, 60, 0, 0, 3, 3, 0, 0, 4, 116, 1, 0, 4, 38, 0, 0, 3, 1, 0, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 59 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow desde el JSON expandido...


...


Construyendo el modelo en TensorFlow desde el JSON expandido...
37/37 [==============================] - 1s 12ms/step
Modelo 243 completado y resultados almacenados.

Procesando modelo 244/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [7, 0, 0, 0, 2, 1, 0, 0, 0, 4, 1, 1, 2, 1, 0, 0, 5, 0, 0, 0, 2, 1, 0, 0, 7, 0, 0, 0, 8, 3, 2, 0, 5, 0, 0, 0, 1, 0, 0, 0, 4, 19, 2, 0, 6, 7, 1, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 244 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow desde el JSON expandido...


c:\Users\herna\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


37/37 [==============================] - 0s 3ms/step
Modelo 244 completado y resultados almacenados.

Procesando modelo 245/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [1, 0, 0, 0, 8, 2, 2, 0, 3, 2, 0, 0, 8, 3, 2, 0, 1, 0, 0, 0, 3, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 6, 6, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 7, 0, 0, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 245 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow desde el JSON expandido...
37/37 [==============================] - 1s 13ms/step
Modelo 245 completado y resultados almacenados.

Procesando modelo 246/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [8, 2, 1, 0, 8, 3, 2, 0, 0, 8, 1, 0, 0, 10, 1, 1, 3, 1, 0, 0, 6, 8, 0, 3, 4, 50, 0, 0, 5, 0, 0, 0, 3, 1, 0, 0, 6, 13, 0, 0, 3, 1, 0, 0, 5, 0, 0, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 246 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow des

c:\Users\herna\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


37/37 [==============================] - 0s 5ms/step
Modelo 247 completado y resultados almacenados.

Procesando modelo 248/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [0, 14, 0, 1, 5, 0, 0, 0, 2, 0, 0, 0, 7, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 6, 7, 0, 1, 8, 2, 2, 0, 5, 0, 0, 0, 8, 3, 1, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 248 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow desde el JSON expandido...
37/37 [==============================] - 0s 9ms/step
Modelo 248 completado y resultados almacenados.

Procesando modelo 249/250...
Generando y codificando arquitectura aleatoria...
Final Encoded Model: [4, 2, 3, 0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 11, 1, 3, 4, 58, 3, 0, 2, 0, 0, 0, 0, 11, 0, 2, 2, 0, 0, 0, 6, 10, 1, 0, 6, 10, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0]
Reparando y decodificando la arquitectura...
Entrenando modelo 249 sin K-Fold Cross Validation...

Construyendo el modelo en TensorFlow d

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib  # Para guardar los modelos

# Cargar los datos
data = pd.read_csv('./model_results_250_50.csv')

# Convertir la columna 'Encoded Architecture' de strings a listas de enteros
data['Encoded Architecture'] = data['Encoded Architecture'].apply(eval)

# Expandir 'Encoded Architecture' en múltiples columnas
X = pd.DataFrame(data['Encoded Architecture'].tolist())
y_metrics = data[['Loss', 'Accuracy', 'Precision', 'Recall', 'F1', 'Specificity']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_metrics, test_size=0.2, random_state=42)

# Diccionario para almacenar modelos y resultados
models = {}
results = {}

# Entrenar un modelo XGBoost para cada métrica en y_metrics
for metric in y_metrics.columns:
    print(f"Entrenando modelo para {metric}...")
    
    # Crear y entrenar el modelo
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    model.fit(X_train, y_train[metric])
    
    # Evaluar el modelo
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test[metric], y_pred)
    print(f"MSE para {metric}: {mse}")
    
    # Guardar el modelo y resultados
    models[metric] = model
    results[metric] = mse

    # Guardar el modelo entrenado en disco
    joblib.dump(model, f"{metric}_surrogate_model.pkl")

print("Modelos entrenados y guardados.")


Entrenando modelo para Loss...
MSE para Loss: 23.083759998043448
Entrenando modelo para Accuracy...
MSE para Accuracy: 0.004411367683333119
Entrenando modelo para Precision...
MSE para Precision: 0.10011947601730091
Entrenando modelo para Recall...
MSE para Recall: 0.13314729103012565
Entrenando modelo para F1...
MSE para F1: 0.09112849451760346
Entrenando modelo para Specificity...
MSE para Specificity: 0.0880968583336883
Modelos entrenados y guardados.


In [33]:
import numpy as np
import joblib
import pandas as pd

# Cargar todos los modelos de métricas
metric_models = {}
metrics = ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1', 'Specificity']

for metric in metrics:
    metric_models[metric] = joblib.load(f"{metric}_surrogate_model.pkl")

# Lista para almacenar resultados de predicciones
results = []

# Generar 10 arquitecturas y predecir métricas para cada una
for i in range(10):
    print(f"\nGenerando arquitectura {i+1}...")
    
    # Generar una arquitectura de ejemplo
    model_dict = generate_random_architecture()
    encoded_model = encode_model_architecture(model_dict, max_alleles=48)
    fixed_model = fixArch(encoded_model, verbose=True)
    
    # Preparar arquitectura para predicción
    example_architecture = np.array(fixed_model).reshape(1, -1)
    print(f"Arquitectura de ejemplo: {example_architecture}")
    
    # Predicciones para cada métrica
    predictions = {'Architecture': fixed_model}
    for metric, model in metric_models.items():
        predicted_value = model.predict(example_architecture)[0]
        predictions[metric] = predicted_value
        print(f"Predicción de {metric} para la arquitectura {i+1}: {predicted_value}")
    
    # Agregar predicciones al conjunto de resultados
    results.append(predictions)

# Crear un DataFrame de pandas con los resultados
df_results = pd.DataFrame(results)
print("\nTabla de predicciones para las 10 arquitecturas generadas:")
(df_results)



Generando arquitectura 1...
Final Encoded Model: [4, 53, 2, 0, 1, 0, 0, 0, 2, 1, 0, 0, 7, 0, 0, 0, 0, 8, 1, 3, 2, 1, 0, 0, 0, 9, 1, 0, 8, 3, 1, 0, 8, 2, 2, 0, 2, 0, 0, 0, 3, 2, 0, 0, 3, 3, 0, 0]
Arquitectura de ejemplo: [[ 4 53  2  0  1  0  0  0  2  1  0  0  7  0  0  0  0  8  1  3  2  1  0  0
   0  9  1  0  8  3  1  0  8  2  2  0  2  0  0  0  3  2  0  0  3  3  0  0]]
Predicción de Loss para la arquitectura 1: 0.7078891396522522
Predicción de Accuracy para la arquitectura 1: 0.5194331407546997
Predicción de Precision para la arquitectura 1: 0.4024452865123749
Predicción de Recall para la arquitectura 1: 0.426135778427124
Predicción de F1 para la arquitectura 1: 0.12071824073791504
Predicción de Specificity para la arquitectura 1: 0.6441644430160522

Generando arquitectura 2...
Final Encoded Model: [8, 2, 2, 0, 2, 1, 0, 0, 8, 1, 2, 0, 4, 60, 3, 0, 4, 64, 2, 0, 8, 1, 2, 0, 5, 0, 0, 0, 4, 53, 2, 0, 2, 1, 0, 0, 1, 0, 0, 0, 8, 1, 2, 0, 8, 3, 1, 0]
Ajustando alcance de repetición de 2 a 0 de

,Architecture,Loss,Accuracy,Precision,Recall,F1,Specificity
0,"[4, 53, 2, 0, 1, 0, 0, 0, 2, 1, 0, 0, 7, 0, 0,...",0.707889,0.519433,0.402445,0.426136,0.120718,0.644164
1,"[8, 0, 2, 0, 2, 1, 0, 0, 8, 1, 2, 0, 4, 60, 3,...",0.903181,0.615736,0.514590,0.496720,0.405106,0.637503
2,"[0, 16, 1, 0, 8, 1, 1, 0, 7, 0, 0, 0, 7, 0, 0,...",0.707626,0.558946,0.599831,0.189385,0.332846,0.808533
3,"[4, 39, 0, 0, 0, 5, 1, 2, 7, 0, 0, 0, 7, 0, 0,...",0.706128,0.552180,0.438445,-0.008633,0.393092,0.730487
4,"[0, 8, 1, 0, 6, 5, 0, 3, 3, 0, 0, 0, 8, 2, 2, ...",0.648998,0.562368,0.165253,0.503448,0.432671,0.845927
5,"[7, 0, 0, 0, 0, 12, 1, 1, 7, 0, 0, 0, 7, 0, 0,...",0.671399,0.635127,0.256700,0.044442,0.418566,0.847529
6,"[0, 8, 0, 3, 1, 0, 0, 0, 1, 0, 0, 0, 7, 0, 0, ...",0.620687,0.679799,0.632293,0.405587,0.412461,0.738959
7,"[8, 0, 1, 0, 0, 15, 1, 1, 6, 16, 1, 1, 8, 2, 1...",1.328857,0.533956,-0.027273,0.058215,0.153296,0.920766
8,"[7, 0, 0, 0, 6, 5, 1, 1, 4, 1, 3, 0, 3, 3, 0, ...",0.674035,0.541820,0.075104,0.149163,0.051117,0.844833
9,"[4, 22, 1, 0, 8, 1, 1, 0, 2, 1, 0, 0, 0, 10, 0...",0.695380,0.507985,0.472584,0.384342,0.241530,0.774944
